# 1D Convolution on FPGA

## 1. Set-up 

In [ ]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username    
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [ ]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

In [ ]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

In [ ]:
!ls {PROJECT_ROOT}

## 2. Install TVM and VTA Simulator

In [ ]:
! gsutil cp "gs://tvm-fcrc-binaries-7f775516ff9dfab922c304049f294cec/tvm.tar.gz" /tmp/tvm.tar.gz
! mkdir -p /tvm
! tar -xf /tmp/tvm.tar.gz --strip-components=4 --directory /tvm
! ls -la /tvm
! bash /tvm/package.sh

## 3. Implement `make_conv1d_fpga_*` functions in `src.ops`

In that function, you are required to implemented 1D convolution and use TVM to optimize it on a simulator of FPGA.
Let $x \in \mathbb{R}^m$ and $y \in \mathbb{R}^n$, then 
$$
\operatorname{conv1d}(x, y)_i = \sum_{j=-\infty}^{\infty} x[j]y[i-j], \forall i \in \{0, 1, \dots, m + n - 1\}
$$

Please use zero padding and unit stride. Please see the numpy convolution function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html).

The `make_conv1d_gpu_scheduler` takes $m$ and $n$, which are the size of the two 1D input array. 
You should return both the TVM scheduler and the TVM opterator for 
1. Input $x$
2. Input $y$
3. Output $out$
Other than these, you should also return the environment and the remote object as already provided in the template code.
These objects will be useful for the next function `make_conv1d_fpga_function`.

The `make_conv1d_fpga_function` will provide means to use the numpy arrays to run in the FPGA simulator. 
Specifically, it will take the information output by the `make_conv1d_gpu_scheduler` and return a function `f` which takes two numpy array and compute the output numpy array.

NOTE: the computation inside `f` must use the scheduler output from `make_conv1d_fpga_scheduler`. 
You shouldn't replace the computation with a numpy or torch library call.

In [ ]:
# Add TVM to the Python path.
import sys
sys.path.append('/tvm/python')
sys.path.append('/tvm/topi/python')
sys.path.append('/tvm/nnvm/python')
sys.path.append('/tvm/vta/python')

In [ ]:
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops_fpga import make_conv1d_fpga_scheduler
from src.ops_fpga import make_conv1d_fpga_function

M = 16384
N = 32
dtype = 'float32'
a_np = np.random.rand(M).astype(dtype)
w_np = np.random.rand(N).astype(dtype)
b_np = np.convolve(a_np, w_np)

info = make_conv1d_fpga_scheduler(M, N)
f = make_conv1d_fpga_function(info)
b_out = f(a_np, w_np)

print("Answer:", b_np)
print("Output:", b)

In [ ]:
import vta
s = info['s']
A = info['input_A']
W = info['input_B']
O = info['output_C']
print(vta.lower(s, [A, W, O], simple_mode=True))

In [ ]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_1dconv_fpga.py